In [11]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [12]:
from util import initialize
AI_MODEL = initialize()

import asyncio
from typing import List, Dict
from pprint import pprint, pformat

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o
Enabling logfire


In [13]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = (await agent.run(f"{prompt}\nInput: {result}")).data
        print(result)
    return result

In [14]:
data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",
    
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",
    
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",
    
    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

print("\nInput text:")
print(report)
formatted_result = await chain(report, data_processing_steps)
print(formatted_result)


Input text:

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.



Logfire project URL: ]8;id=700369;https://logfire.pydantic.dev/intellectronica/buildingwithai\https://logfire.pydantic.dev/intellectronica/buildingwithai]8;;\


Step 1:
14:19:46.978 agent run prompt=Extract only the numerical values and their associated metrics...tisfaction is at 87 points.
Operating margin improved to 34%.

14:19:46.979   preparing model and tools run_step=1
14:19:46.979   model request
14:19:48.457   handle model response
92: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
43: new user acquisition cost  
78%: product adoption rate  
87: employee satisfaction  
34%: operating margin

Step 2:
14:19:48.458 agent run prompt=Convert all numerical values to percentages where possible.
  ...ption rate  
87: employee satisfaction  
34%: operating margin
14:19:48.459   preparing model and tools run_step=1
14:19:48.459   model request
14:19:49.426   handle model response
92%: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
43%: new user acquisition cost  
78%: product adoption rate  
87%: employee satisfaction  
34%: operating margin

Step 3:
14:19:49.427 a

In [15]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(
        AI_MODEL,
        system_prompt=prompt,
    )
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.data for result in results]

In [16]:
stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

impact_results = await parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders
)

for result in impact_results:
    print(result)

14:19:52.099 agent run prompt=Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns
14:19:52.100   preparing model and tools run_step=1
14:19:52.100   model request
14:19:52.111 agent run prompt=Employees:
    - Job security worries
    - Need new skills
    - Want clear direction
14:19:52.112   preparing model and tools run_step=1
14:19:52.112   model request
14:19:52.123 agent run prompt=Investors:
    - Expect growth
    - Want cost control
    - Risk concerns
14:19:52.123   preparing model and tools run_step=1
14:19:52.124   model request
14:19:52.133 agent run prompt=Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions
14:19:52.134   preparing model and tools run_step=1
14:19:52.135   model request
             agent run prompt=Employees:
    - Job security worries
    - Need new skills
    - Want clear direction
14:19:57.189   handle model response
             agent run prompt=Investors:
    - Expect growth
    - Want 

In [17]:
async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    print(f"\nAvailable routes: {list(routes.keys())}")

    class RouteSelection(BaseModel):
        reasoning: str = Field(..., description=(
            'Brief explanation of why this ticket should be routed to a specific team. '
            'Consider key terms, user intent, and urgency level.'
        ))
        selection: str = Field(..., description='The chosen team name')
    
    route_response = await Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team from these options: '
            f'{list(routes.keys())}'
        ),
        result_type=RouteSelection,
    ).run(input)
    reasoning = route_response.data.reasoning
    route_key = route_response.data.selection.strip().lower()
    
    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")
    
    # Process input with selected specialized prompt
    return (await Agent(
        AI_MODEL,
        system_prompt=routes[route_key],
    ).run(input)).data

In [18]:
support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.
    
    Input: """,
    
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.
    
    Input: """,
    
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.
    
    Input: """,
    
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone.
    
    Input: """
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = await route(ticket, support_routes)
    print(response)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
14:19:59.525 agent run prompt=[Scrubbed due to 'password']
14:19:59.525   preparing model and tools run_step=1
14:19:59.526   model request
14:20:00.661   handle model response
Routing Analysis:
The user is facing an issue with logging into their account and receiving an 'invalid password' error message. This issue is most appropriate for the 'account' team, as they specialize in handling problems related to account access and login credentials. The urgency mentioned by the user about submitting a repor